In [1]:
import argparse
import tensorflow as tf
from core.yolov3 import YOLOV3
from core.config import cfg

/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
org_weights_path = cfg.YOLO.ORIGINAL_WEIGHT
cur_weights_path = cfg.YOLO.DEMO_WEIGHT
preserve_cur_names = ['conv_sbbox', 'conv_mbbox', 'conv_lbbox']
preserve_org_names = ['Conv_6', 'Conv_14', 'Conv_22']
print(org_weights_path)
print(cur_weights_path)

./checkpoint/yolov3_coco.ckpt
./checkpoint/yolov3_coco_demo.ckpt


In [5]:
org_weights_mess = []
tf.Graph().as_default()
load = tf.train.import_meta_graph(org_weights_path + '.meta')

In [6]:
load

In [7]:
with tf.Session() as sess:
    load.restore(sess, org_weights_path)

W0822 14:03:25.660254 4637427136 deprecation.py:323] From /Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [9]:
len(tf.global_variables())

366

In [10]:
tf.global_variables()[0:5]

[<tf.Variable 'yolov3/darknet-53/Conv/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'yolov3/darknet-53/Conv/BatchNorm/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'yolov3/darknet-53/Conv/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'yolov3/darknet-53/Conv/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'yolov3/darknet-53/Conv/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>]

### Test train.py

In [1]:
import os
import time
import shutil
import numpy as np
import tensorflow as tf
import core.utils as utils
from tqdm import tqdm
from core.dataset import Dataset
from core.yolov3 import YOLOV3
from core.config import cfg

/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
tmp_anchor_per_scale    = cfg.YOLO.ANCHOR_PER_SCALE
tmp_classes             = utils.read_class_names(cfg.YOLO.CLASSES)
tmp_num_classes         = len(tmp_classes)
tmp_learn_rate_init     = cfg.TRAIN.LEARN_RATE_INIT
tmp_learn_rate_end      = cfg.TRAIN.LEARN_RATE_END
tmp_first_stage_epochs  = cfg.TRAIN.FISRT_STAGE_EPOCHS
tmp_second_stage_epochs = cfg.TRAIN.SECOND_STAGE_EPOCHS
tmp_warmup_periods      = cfg.TRAIN.WARMUP_EPOCHS
tmp_initial_weight      = cfg.TRAIN.INITIAL_WEIGHT
tmp_time                = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(time.time()))
tmp_moving_ave_decay    = cfg.YOLO.MOVING_AVE_DECAY
tmp_max_bbox_per_scale  = 150
tmp_train_logdir        = "./data/log/train"
tmp_trainset            = Dataset('train')
tmp_testset             = Dataset('test')
tmp_steps_per_period    = len(tmp_trainset)
tmp_sess                = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))

In [9]:
with tf.name_scope('define_input'):
    tmp_input_data   = tf.placeholder(dtype=tf.float32, name='input_data')
    # the label of the objects in each box
    tmp_label_sbbox  = tf.placeholder(dtype=tf.float32, name='label_sbbox')
    tmp_label_mbbox  = tf.placeholder(dtype=tf.float32, name='label_mbbox')
    tmp_label_lbbox  = tf.placeholder(dtype=tf.float32, name='label_lbbox')
    
    # the coordinates of the boxes
    tmp_true_sbboxes = tf.placeholder(dtype=tf.float32, name='sbboxes')
    tmp_true_mbboxes = tf.placeholder(dtype=tf.float32, name='mbboxes')
    tmp_true_lbboxes = tf.placeholder(dtype=tf.float32, name='lbboxes')
    tmp_trainable     = tf.placeholder(dtype=tf.bool, name='training')